<a href="https://colab.research.google.com/github/OmniaOsama03/RESTful-API-Web-Service/blob/main/CSC406_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string

from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# Load the dataset
data = pd.read_csv("Sustainability_QA_List.csv")

#Displaying last 8 rows
data.head(10)

,Question,Answer
0,What are some everyday practices to reduce my ...,"Reduce energy consumption, use public transpor..."
1,How can I conserve water at home in a cost-eff...,"Install water-efficient fixtures, fix leaks, a..."
2,What are the benefits of using renewable energ...,"Renewable energy is cleaner, reduces emissions..."
3,Can you explain the concept of sustainable agr...,Sustainable agriculture involves practices tha...
4,What are some examples of sustainable transpor...,"Biking, walking, carpooling, and public transi..."
5,How can recycling impact environmental sustain...,"Recycling reduces waste in landfills, conserve..."
6,What is the significance of reducing single-us...,"It helps reduce pollution, save resources, and..."
7,How can businesses adopt more sustainable prac...,"Implement recycling programs, reduce energy us..."
8,What are some tips for sustainable grocery sho...,"Buy bulk, choose local and seasonal products, ..."
9,Can you explain the importance of local sourci...,Reduces transportation emissions and supports ...


In [ ]:
!pip install gradientai --upgrade

In [ ]:
import os
os.environ['GRADIENT_ACCESS_TOKEN'] = "cVc33fsF5XVKcNVaEpgDfpFbKLfsQX5I"
os.environ['GRADIENT_WORKSPACE_ID'] = "ba1a0084-82f6-42d1-befd-a46d7b1ebc83_workspace"

In [ ]:
!pip install gradio==3.48.0
import gradio as gr
from gradientai import Gradient


The Model before text preprocessing & Fine-tuning

In [ ]:
import gradio as gr
from gradientai import Gradient

def CreateBaseModel():
    with Gradient() as gradient:
        # Get the base model
        base_model = gradient.get_base_model(base_model_slug = "nous-hermes2")

        Grp2_baseModel = base_model.create_model_adapter(
            name="test"
        )


        def get_base_model_response(userQuery):
            MyOutput = Grp2_baseModel.complete(query = userQuery, max_generated_token_count = 300).generated_output
            return MyOutput

        gr.Interface(fn=get_base_model_response,
                     outputs="text",
                     title="Base Model Chatbot",
                     inputs="text",
                     description="Interact with the base model. Try asking something!").launch()

CreateBaseModel()

IMPORTANT: You are using gradio version 3.48.0, however version 4.29.0 is available, please upgrade.
--------
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6d3f04780ba7aec3dd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


The Model after text preprocessing & Fine-tuning

In [ ]:
import gradio as gr
import pandas as pd
import re
import string

# Assuming data is already read from the CSV
data = pd.read_csv("Sustainability_QA_List.csv")

def preprocess_text(text):
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    #Regex Processing
    text = re.sub(r'\b\w*\d+\w*\b', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\b\w{1,2}\b', '', text)
    text = re.sub(r'\b\w*_\b', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def CreateBaseModel():
    with Gradient() as gradient:
        base_model = gradient.get_base_model(base_model_slug="nous-hermes2")

        Grp2_NewModel = base_model.create_model_adapter(
            name="test model"
        )
        print(f"Created model adapter with id {Grp2_NewModel.id}")

        MyPrompts = []

        # Iterate through the DataFrame rows
        for index, row in data.iterrows():
            instruction = preprocess_text(row['Question']) #HERE HERE HERE
            response = preprocess_text(row['Answer'])
            formatted_dict = {
                "inputs": f"### Instruction:{instruction}  \n\n### Response: {response}"
            }
            MyPrompts.append(formatted_dict)

        # Fine-tuning the model
        num_epochs = 2
        for count in range(num_epochs):
            print(f"Fine-tuning the model, iteration {count + 1}")
            Grp2_NewModel.fine_tune(samples=MyPrompts)

        def get_model_response(userQuery):
            userQuery = preprocess_text(userQuery) #HERE HERE HERE
            MyNewOutput = Grp2_NewModel.complete(query=userQuery, max_generated_token_count=400).generated_output
            return MyNewOutput

        gr.Interface(
            fn=get_model_response,
            outputs="text",
            title="Fine-tuned Chatbot",
            inputs="text",
            description="This model has been fine-tuned with specific knowledge. Try asking something!"
        ).launch()

CreateBaseModel()

Created model adapter with id 5e9e24d1-f911-4f4e-bafc-8c1e9109f6cd_model_adapter
Fine-tuning the model, iteration 1
Fine-tuning the model, iteration 2
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.48.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://8610f48785f1edca29.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
